In [1]:
import utilities
import matplotlib.pyplot as plt
import numpy as np
import AdaBoost
import KernelPerceptron

In [2]:
height = width = 16
class_number = 10

In [3]:
train_data = 'Datasets/full_dataset.dat'

In [4]:
data,labels = utilities.read_data(train_data,width,height)

In [5]:
train_data,test_data,train_labels,test_labels = utilities.data_split(data,labels,0.3)
print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(1446, 256)
(620, 256)
(1446,)
(620,)


In [6]:
data_per_class_dictionary = utilities.collect_each_class_images(train_data,train_labels,10)
pair_datasets = utilities.create_1_VS_1_dataset(data_per_class_dictionary)
print("Paired classifiers number: " + str(len(pair_datasets)))

282 images of class 0 found
222 images of class 1 found
153 images of class 2 found
98 images of class 3 found
104 images of class 4 found
80 images of class 5 found
129 images of class 6 found
135 images of class 7 found
112 images of class 8 found
131 images of class 9 found
45


In [7]:
#algorithm = 'AdaBoost'
#parameters = [50]
algorithm = 'KernelPerceptron'
parameters = ['gaussian',0.5]
models_dict = utilities.one_VS_all_training(data_per_class_dictionary,class_number,algorithm,parameters)
#pair_models = utilities.one_vs_one_training(pair_datasets,algorithm,parameters)

Training a classifier for: 0
Training a classifier for: 1
Training a classifier for: 2
Training a classifier for: 3
Training a classifier for: 4
Training a classifier for: 5
Training a classifier for: 6
Training a classifier for: 7
Training a classifier for: 8
Training a classifier for: 9


In [8]:
test_predictions = utilities.one_VS_all_testing(models_dict,test_data,test_labels)
#test_predictions = utilities.one_vs_one_testing(pair_models,test_data,class_number)

In [9]:
print(utilities.calculate_accuracy(test_predictions,test_labels))

91.77419354838709
